# Chemical equilibrium with fixed pH


In this tutorial we demonstrate how chemical equilibrium calculations with a
fixed pH value can be accomplished in Reaktoro.

We'll consider a relatively simple chemical description of an aqueous solution.
This will permit us to describe the concepts and implications of such pH
constraint on the computed equilibrium state.

Let's then create a chemical system with just a single aqueous phase containing
some selected species (instead of all possible species in the database with a
given list of elements). For this phase, we use a Debye–Hückel activity model.
See code below:

In [17]:
import reaktoro as rkt
import numpy as np

db = rkt.SupcrtDatabase("supcrtbl")

solution = rkt.AqueousPhase("H2O(aq) H+ OH- Na+ Cl- HCO3- CO2(aq) CO3-2")
solution.setActivityModel(rkt.ActivityModelDebyeHuckel())

system = rkt.ChemicalSystem(db, solution)

We want to perform a chemical equilibrium calculation in which the following
properties are constrained:

* temperature;
* pressure; and
* pH;

Let's create a dedicated and optimized chemical equilibrium solver to deal with
these types of problems. 

First, we create an object of class {{EquilibriumSpecs}}, which we use to provide
the equilibrium specifications for our solver later on:

In [18]:
specs = rkt.EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.pH()

```{note}
By specifying that pH is constrained, we will be constructing later an
equilibrium solver that presumes that the system is open to H<sup>+</sup>. Thus,
during the calculation, a certain amount of H<sup>+</sup> will be added or removed
from the system so that the pH constraint is satisfied. Because H<sup>+</sup> is a
charged species, this will most likely produce a chemical state with non-zero
electrical charge. If zero electrical charge is expected, then a charge balance
constraint must be enforced, and the system needs to be additionally open to a
negative ion such as Cl<sup>-</sup>. We'll demonstrate how this is accomplished
later.
```

Once we have created this specifications object, we can now create our
equilibrium solver:

In [19]:
solver = rkt.EquilibriumSolver(specs)

Let's now create an initial chemical state for our chemical system:

In [20]:
state = rkt.ChemicalState(system)
state.temperature(30.0, "celsius")
state.pressure(1.0, "bar")
state.set("H2O(aq)", 1.0, "kg")
state.set("Na+", 0.01, "mol")
state.set("Cl-", 0.01, "mol")
state.set("CO2(aq)", 0.1, "mol")

This state is not in chemical equilibrium. Before we equilibrate it
with our equilibrium solver, let's create a copy of the vector of element
amounts in this state, which we call `b0` below. We do this because we will
compare the amounts of each element before and after equilibration, in which
temperature, pressure, and pH are prescribed.

In [21]:
b0 = state.elementAmounts().asarray()

Let's now define the temperature, pressure, and pH conditions we want to impose
at equilibrium for this aqueous solution currently in disequilibrium. We do
this by using an object of class {{EquilibriumConditions}}:

In [22]:
conditions = rkt.EquilibriumConditions(specs)
conditions.temperature(50.0, "celsius")
conditions.pressure(10.0, "bar")
conditions.pH(2.0)

We have everything we need now to perform the equilibrium calculation:

* an equilibrium solver (the `solver` object of class {{EquilibriumSolver}});
* a chemical state to equilibrate (the `state` object of class {{ChemicalState}}); and
* the conditions we are imposing for this equilibrium state (the `conditions`
  object of class {{EquilibriumConditions}}).

The code below will perform the equilibrium calculation and modify the `state`
object to reflect the computed chemical equilibrium state:

In [23]:
result = solver.solve(state, conditions)

It's always advisable to verify if the calculation succeeded:

In [24]:
print("Successful computation!" if result.optima.succeeded else "Computation has failed!")

Successful computation!


Let's check the computed chemical equilibrium state: 

In [25]:
print(state)

+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |      323.15 |    K |
| Pressure        |       1e+06 |   Pa |
| Element Amount: |             |      |
| :: H            |     111.028 |  mol |
| :: C            |         0.1 |  mol |
| :: O            |     55.7084 |  mol |
| :: Na           |        0.01 |  mol |
| :: Cl           |        0.01 |  mol |
| Species Amount: |             |      |
| :: H2O(aq)      |     55.5084 |  mol |
| :: H+           |   0.0110073 |  mol |
| :: OH-          | 6.03742e-12 |  mol |
| :: Na+          |        0.01 |  mol |
| :: Cl-          |        0.01 |  mol |
| :: HCO3-        | 6.05697e-06 |  mol |
| :: CO2(aq)      |   0.0999939 |  mol |
| :: CO3-2        | 5.72532e-14 |  mol |
+-----------------+-------------+------+


Well, there is no pH in this table because {{ChemicalState}} objects only store
temperature, pressure, and species amounts. 

What we need now is an object of class {{AqueousProps}}. This class was
specifically designed to inspect thermodynamic and chemical properties related
to aqueous solutions. Let's create an {{AqueousProps}} object and print it:

In [26]:
aprops = rkt.AqueousProps(state)
print(aprops)

+--------------------------+-------------+-------+
| Property                 |       Value |  Unit |
+--------------------------+-------------+-------+
| Temperature              |      323.15 |     K |
| Pressure                 |       1e+06 |    Pa |
| Ionic Strength (Effect.) |   0.0155067 | molal |
| Ionic Strength (Stoich.) |   0.0155067 | molal |
| pH                       |           2 |       |
| pE                       |    -1.34192 |       |
| Eh                       |  -0.0860438 |     V |
| Element Molality:        |             |       |
| :: C                     |         0.1 | molal |
| :: Na                    |        0.01 | molal |
| :: Cl                    |        0.01 | molal |
| Species Molality:        |             |       |
| :: H+                    |   0.0110073 | molal |
| :: OH-                   | 6.03743e-12 | molal |
| :: Na+                   |        0.01 | molal |
| :: Cl-                   |        0.01 | molal |
| :: HCO3-                 | 6.

Great! We can now see that pH is exactly 2, as we enforced, and temperature and
pressure are 50 °C and 10 bar (but displayed in the table as 323.15 K and
10<sup>6</sup> Pa respectively).

Let's now compare the amounts of elements at equilibrium state with that at the
initial state. 

In [27]:
b = state.elementAmounts().asarray()

for i, element in enumerate(system.elements()):
    print(f"Δb[{element.symbol()}] = {b[i] - b0[i]}")

Δb[H] = 0.011001266746177407
Δb[C] = 1.4155343563970746e-15
Δb[O] = 0.0
Δb[Na] = 0.0
Δb[Cl] = 0.0


Note elements Na, Cl, C, and O are numerically conserved (i.e., their final
and initial amounts are zero or close to machine precision, 
approximately 1.11e-16). This conservation is not observed for element H, however.

Let's check the electrical charge of the state:

In [28]:
print(state.charge())

0.0110013


Recall that specifying pH implies that the system is open to H<sup>+</sup> during the
chemical equilibrium calculation. Thus, the values above for element H and
electric charge reflect this behavior. It indicates that H<sup>+</sup> was added into
the system to obtain an aqueous state with the stipulated pH value.

Let's now create an equilibrium solver that considers not only temperature,
pressure, and pH constraints, but also an additional charge constraint. We want
electrical charge balance to be attained by adding/removing Cl<sup>-</sup> as much as
needed. Because of this, we specify below in the {{EquilibriumSpecs}} object that
the system is open to Cl<sup>-</sup>.

```{note}
The formulation below equivalent to the task of finding an aqueous solution
state in which a desired pH is obtained by titrating HCl.
```

In [35]:
specs = rkt.EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.pH()
specs.charge()
specs.openTo("Cl-")

solver = rkt.EquilibriumSolver(specs)

conditions = rkt.EquilibriumConditions(specs)
conditions.temperature(50.0, "celsius")
conditions.pressure(10.0, "bar")
conditions.pH(2.0)
conditions.charge(0.0)

solver.solve(state, conditions)


+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |      323.15 |    K |
| Pressure        |       1e+06 |   Pa |
| Element Amount: |             |      |
| :: H            |     111.028 |  mol |
| :: C            |         0.1 |  mol |
| :: O            |     55.7084 |  mol |
| :: Na           |        0.01 |  mol |
| :: Cl           |    0.021127 |  mol |
| Species Amount: |             |      |
| :: H2O(aq)      |     55.5084 |  mol |
| :: H+           |   0.0111332 |  mol |
| :: OH-          | 6.13581e-12 |  mol |
| :: Na+          |        0.01 |  mol |
| :: Cl-          |    0.021127 |  mol |
| :: HCO3-        | 6.15075e-06 |  mol |
| :: CO2(aq)      |   0.0999938 |  mol |
| :: CO3-2        | 6.07025e-14 |  mol |
+-----------------+-------------+------+


The new equilibrium state is:

In [ ]:

print(state)

It's electrical charge should be zero or close to machine precision:

In [36]:
print(state.charge())

-2.23045e-16


And its pH should be 2:

In [37]:
aprops = rkt.AqueousProps(state)
print(aprops.pH())

2


This is the end of this tutorial. We have demonstrated here how to perform
chemical equilibrium calculations with fixed pH with and without electrical
charge balance. 